## Task 1. Disparity Map

1. Calculate a disparity map for the left image using classical methods. Create an image of floating point numbers that estimate the disparity (x-x’) for every pixel in the left image.

In [1]:
import os
import numpy as np  # cupy
import cv2
import time
import sys
import matplotlib.pyplot as plt

rootpath = './'
# initialize the lowest impossible value for ncc and largest impossible value for ssd
IMPOSSIBLE_NCC= sys.float_info.min  # min float value in python
IMPOSSIBLE_SSD= sys.float_info.max  # max

# Read in images from a filepath as graycsale.
def load_images(img_path):
    return cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
#     return cv2.imread(os.path.join(rootpath, img_path), cv2.IMREAD_GRAYSCALE)

# This function addresses: Are all pixels in the matching window equally important?
# Compute the pixel values of maching windows weighted by their euclidian distance to the central pixel
# so for each pixel: value = value/d, where d= sqrt((x2-x1)^2 + (y2-y1)^2)
# TODO: try other distance formulas or diff ways of reweighting (i.e. x(BLOCK_SIZE-euc_dis))
def compute_weighted_block_pixels(block,BLOCK_SIZE):
    cx, cy = (int((BLOCK_SIZE-1)/2), int((BLOCK_SIZE-1)/2))  # recompute central pt coordinates within block
    x_coors, y_coors = np.meshgrid(np.arange(BLOCK_SIZE), np.arange(BLOCK_SIZE))# x,y coordinate matrices of the block
    # get matrix of euclidian distances relative to the central pt
    euc_dis = np.sqrt((x_coors - cx)**2 + (y_coors - cy)**2)  
    euc_dis[cx,cy]= 1 # replace 0 with 1 for central pt
    # penalize closest neighbors with arbitrary distance= 1.1  tune
    euc_dis[cx-1,cy]= 1.1 
    euc_dis[cx+1,cy]= 1.1 
    euc_dis[cx,cy-1]= 1.1 
    euc_dis[cx,cy+1]= 1.1 
    block= np.multiply(block, 1/euc_dis)  # element-wise multiplication
    return block

# compute NCC (normalized corss correlation) between two blocks
def compute_ncc(blockL, blockR, BLOCK_SIZE,PENALIZE_PIXEL):
    if (blockL.shape != blockR.shape):  
        return IMPOSSIBLE_NCC 
    if (PENALIZE_PIXEL):
        blockL = compute_weighted_block_pixels(blockL, BLOCK_SIZE)    
        blockR = compute_weighted_block_pixels(blockR, BLOCK_SIZE)
    blockL= blockL.astype(np.float64) # prevent int_scalar overflow
    blockR= blockR.astype(np.float64)
    dividend= np.sum(np.multiply(blockL,blockR))
    divisor= np.sqrt(np.sum(blockL** 2)*np.sum(blockR** 2))
    return dividend/divisor

# compute ZNCC (Zero Mean Normalized Cross-Correlation) between two blocks
def compute_zncc(blockL, blockR, BLOCK_SIZE,PENALIZE_PIXEL):
    blockL_mean = np.sum(blockL)/BLOCK_SIZE  # compute local mean
    blockR_mean = np.sum(blockR)/BLOCK_SIZE 
    blockL_sub = blockL-blockL_mean  # block subtracted with local mean
    blockR_sub = blockR-blockR_mean
    zncc = compute_ncc(blockL_sub, blockR_sub, BLOCK_SIZE,PENALIZE_PIXEL)
    return zncc

def compute_ssd(blockL, blockR, BLOCK_SIZE,PENALIZE_PIXEL):
    # TODO: compute sd over the whole image as a matrix operation, 
    # then sum using the sliding window technique
    if (blockL.shape != blockR.shape):  
        return IMPOSSIBLE_SSD  
    if (PENALIZE_PIXEL):
        blockL = compute_weighted_block_pixels(blockL, BLOCK_SIZE)    
        blockR = compute_weighted_block_pixels(blockR, BLOCK_SIZE)
    blockL= blockL.astype(np.float64) # prevent int_scalar overflow
    blockR= blockR.astype(np.float64)
    ssd = np.sum((blockL - blockR) ** 2)
    return ssd

# Put a sliding window along the coordiantes of blockL in imageR and get multiple candidate blockRs
# select the one with smallest SSD or largest NCC
def find_matched_block(imgR, ptL_x, ptL_y, SLIDING_WINDOW_SIZE, blockL,PENALIZE_PIXEL):
    EXTREME_METRIC = IMPOSSIBLE_SSD   # set for ncc/ssd
    best_metric = EXTREME_METRIC  
    BLOCK_SIZE = blockL.shape[0]
    h, w = imgR.shape
    
    best_block_x = 0 # initialize x coordinate of best matching blockR
    # range of x coordinate of the upper left edge on blockR 
    ptR_x_min = max(0, ptL_x - SLIDING_WINDOW_SIZE)
    ptR_x_max = min(w, ptR_x_min + 2 * SLIDING_WINDOW_SIZE)  

    # compute blockR in diff positions and get the one with minimum matching cost
    for x in range(ptR_x_min, ptR_x_max):
        blockR = imgR[ptL_y : ptL_y + BLOCK_SIZE, x : x + BLOCK_SIZE] # no shift vertically- use y coor from blockL
        metric = compute_ssd(blockL,blockR,BLOCK_SIZE,PENALIZE_PIXEL)
        if (metric < best_metric):  #  < for ssd, > for ncc. 
            best_metric = metric
            best_block_x = x
    if (best_metric == EXTREME_METRIC):  # if not find the best matching block
        return ptL_x  
    else:
        return best_block_x  

  
# Computing stereo disparity using the block matching algorithm.
def compute_disparity(imgL, imgR, BLOCK_SIZE, SLIDING_WINDOW_SIZE, PENALIZE_PIXEL):
    h, w = imgL.shape
    disparity_values = np.zeros((h, w))  #init
    
    if (PENALIZE_PIXEL==False):  # if not reweight pixels
        for x in range(0, w - BLOCK_SIZE):  
            for y in range(0, h - BLOCK_SIZE):
                blockL = imgL[y : y + BLOCK_SIZE, x : x + BLOCK_SIZE]  # when indexing, row first, then col  point(y,x)
                blockR_x = find_matched_block(imgR, x, y,SLIDING_WINDOW_SIZE, blockL,False)
                disparity_values[y, x]= abs(x - blockR_x)  # (x-x’)
   
    else:  # if reweight pixels
        block_cx= int((BLOCK_SIZE-1)/2)  # x coor of block center
        block_cy= block_cx
        for x in range(block_cx, w - block_cx):  
            for y in range(block_cy, h - block_cy):
                blockL = imgL[y-block_cy:y + block_cy, x-block_cx : x + block_cx]  
                blockR_x = find_matched_block(imgR, x, y, SLIDING_WINDOW_SIZE, blockL,True)
                disparity_values[y, x]= abs(x - blockR_x)  # (x-x’)
                
    return disparity_values

In [2]:
def display_images(imgL, imgR, imgD, scaled_dis_map):
    plt.subplots(figsize = (15, 15)) 
    plt.subplot(2, 2, 1)
    plt.imshow(imgL, cmap = 'gray')  
    plt.title('Left image')
    plt.axis('off')

    plt.subplot(2, 2, 2)
    plt.imshow(imgR, cmap = 'gray')  
    plt.title('Right image')
    plt.axis('off')

    plt.subplot(2, 2, 3)
    plt.imshow(imgD, 'gray')
    plt.title('Ground truth disparity image')
    plt.axis('off')

    plt.subplot(2, 2, 4)
    plt.imshow(scaled_dis_map, 'gray')
    plt.title('Predicted disparity Image')
    plt.axis('off')
    plt.show()

## Task 2. Statistics Calculation

Compute:
a. The rms (root mean squared) error between the values in your disparity map
and those in the ground truth
b. The fractions of pixels with errors less than 4, 2, 1, 0.5 and 0.25 pixels
c. The runtime of your algorithm in seconds per image

In [3]:
# compute RMSE between disparity image and ground truth
def compute_RMSE(imgD,disparity_map):
    errors=[]
    for x in range(imgD.shape[1]):
        for y in range(imgD.shape[0]):
            if imgD[y][x]!=0: # restrict to pixels with non-zero disparity
                errors.append(imgD[y][x]/256 -disparity_map[y][x]) 
    RMSE= np.sqrt(sum([e ** 2 for e in errors])/len(errors))
    RMSE= round(RMSE,2)
    print("RMSE between disparity map and group truth is ",RMSE)
    return errors, RMSE
            
# compute fractions of pixels with errors less than 4, 2, 1, 0.5 and 0.25 pixels
def compute_error_fractions(error_list):
    fractions=[]
    thresholds=[4,2,1,0.5,0.25]
    for threshold in thresholds:
        errors_under= [abs(e) for e in error_list if abs(e) < threshold]
        fraction= len(errors_under)/len(error_list)
        fraction= round(fraction*100,2) 
        fractions.append(fraction)    
        print("Pixels with errors less than ",threshold," pixels:",fraction, "%")
    return fractions

In [4]:
def main_algorithm(path, BLOCK_SIZE, SLIDING_WINDOW_SIZE, PENALIZE_PIXEL):
    start_time = time.time()
    pathL = path + "left.jpg"
    pathR = path + "right.jpg"
    pathD = path + "disparity.png"
    imgL = load_images(pathL)
    imgR = load_images(pathR)   
    imgD = cv2.imread(pathD, cv2.IMREAD_UNCHANGED) # dont use greyscale to load greyscale img
    imgD = imgD.astype(float)
    # 1. compute the predicted disparity map and the copy for display
    disparity_map = compute_disparity(imgL, imgR, BLOCK_SIZE, SLIDING_WINDOW_SIZE,PENALIZE_PIXEL)
 
    display_disparity_map = disparity_map.copy()
    for y in range(imgD.shape[0]):
        for x in range(imgD.shape[1]):
            if imgD[y][x] == 0:
                display_disparity_map[y][x] = 0

#     display_images(imgL, imgR, imgD, display_disparity_map)   #TODO not display for tuning
    print("BLOCK_SIZE:",BLOCK_SIZE, ", SLIDING_WINDOW_SIZE:",SLIDING_WINDOW_SIZE,", ",PENALIZE_PIXEL)
    
    # 2. calculate statisitcs
    err_list,RMSE = compute_RMSE(imgD, disparity_map)  # RMSE
    percent_list= compute_error_fractions(err_list)  # fraction of errors under thresholds
    end_time = time.time()
    run_time = int(np.floor(end_time - start_time))
    print("Runtime:", run_time, "seconds per image") # runtime
    return RMSE, percent_list[0], run_time  # for 

# path = "Dataset/2018-07-09-16-11-56_2018-07-09-16-55-15-689-"
# main_algorithm(path, 9, 43,False)

In [5]:
# for tuning
# path = "Dataset/2018-07-09-16-11-56_2018-07-09-16-55-15-689-"
# BLOCK_SIZE_LIST= [5,7,9,11,13]  # linear size of matching blocks- ODD # ONLY
# SLIDING_WINDOW_SIZE_LIST= [43,45,47,49,51]     # window of horizontal shift in blockR
# PENALIZE_PIXEL_BOOLEANS= [False]

# def find_optimal_params():
#     RMSE_list= []
#     percent_less_than4= []
#     runtime_list= []
    
#     for block_size in BLOCK_SIZE_LIST:
#         for max_shift in SLIDING_WINDOW_SIZE_LIST:
#             for is_penalized in PENALIZE_PIXEL_BOOLEANS:
#                 RMSE, percent_list, run_time = main_algorithm(path, block_size, max_shift, is_penalized)   
#                 RMSE_list.append(RMSE)
#                 percent_less_than4.append(percent_list[0])
#                 runtime_list.append(run_time)
#     return RMSE_list,percent_less_than4,runtime_list

# RMSE_list,percent_less_than4,runtime_list= find_optimal_params()
# print(RMSE_list)
# print(RMSE_list.index(min(RMSE_list)))
# print(percent_less_than4)
# print(percent_less_than4.index(max(percent_less_than4)))

In [7]:
# retrieve all imgs under folder
img_arr = os.listdir('Dataset/')
suf_arr= ['disparity.png','left.jpg','right.jpg']
unique_arr= []  # unique list of image prefixes
for img in img_arr:  # remove suffixes from image names
    for suf in suf_arr:
        if suf in img:
            unique_arr.append(img.replace(suf,"") )

unique_arr= list(set(unique_arr))
RMSE_list= []
percent_less_than4_list= []
runtime_list= []
for img in unique_arr:
    path= 'Dataset/'+img
    RMSE,percent_less_than4,runtime= main_algorithm(path, 13, 53,False)
    RMSE_list.append(RMSE)
    percent_less_than4_list.append(percent_less_than4)
    runtime_list.append(runtime)

BLOCK_SIZE: 13 , SLIDING_WINDOW_SIZE: 53 ,  False
RMSE between disparity map and group truth is  18.07
Pixels with errors less than  4  pixels: 52.99 %
Pixels with errors less than  2  pixels: 40.26 %
Pixels with errors less than  1  pixels: 29.91 %
Pixels with errors less than  0.5  pixels: 18.22 %
Pixels with errors less than  0.25  pixels: 9.67 %
Runtime: 396 seconds per image
BLOCK_SIZE: 13 , SLIDING_WINDOW_SIZE: 53 ,  False
RMSE between disparity map and group truth is  15.1
Pixels with errors less than  4  pixels: 61.47 %
Pixels with errors less than  2  pixels: 46.4 %
Pixels with errors less than  1  pixels: 32.15 %
Pixels with errors less than  0.5  pixels: 18.62 %
Pixels with errors less than  0.25  pixels: 8.76 %
Runtime: 406 seconds per image
BLOCK_SIZE: 13 , SLIDING_WINDOW_SIZE: 53 ,  False
RMSE between disparity map and group truth is  25.63
Pixels with errors less than  4  pixels: 51.48 %
Pixels with errors less than  2  pixels: 41.03 %
Pixels with errors less than  1  pi

BLOCK_SIZE: 13 , SLIDING_WINDOW_SIZE: 53 ,  False
RMSE between disparity map and group truth is  19.76
Pixels with errors less than  4  pixels: 51.65 %
Pixels with errors less than  2  pixels: 41.93 %
Pixels with errors less than  1  pixels: 29.43 %
Pixels with errors less than  0.5  pixels: 17.6 %
Pixels with errors less than  0.25  pixels: 8.66 %
Runtime: 381 seconds per image
BLOCK_SIZE: 13 , SLIDING_WINDOW_SIZE: 53 ,  False
RMSE between disparity map and group truth is  19.24
Pixels with errors less than  4  pixels: 57.76 %
Pixels with errors less than  2  pixels: 43.14 %
Pixels with errors less than  1  pixels: 29.63 %
Pixels with errors less than  0.5  pixels: 18.95 %
Pixels with errors less than  0.25  pixels: 10.65 %
Runtime: 378 seconds per image
BLOCK_SIZE: 13 , SLIDING_WINDOW_SIZE: 53 ,  False
RMSE between disparity map and group truth is  19.52
Pixels with errors less than  4  pixels: 54.33 %
Pixels with errors less than  2  pixels: 42.33 %
Pixels with errors less than  1  

In [12]:
# print avg RMSE, % pixels, runtime
print("Average RMSE in 25 images: ",sum(RMSE_list)/len(RMSE_list))
print("Average % pixels with error< 4 : ",sum(percent_less_than4_list)/len(percent_less_than4_list))
print("Average runtime (seconds) : ",sum(runtime_list)/len(runtime_list))

Average RMSE in 25 images:  19.523199999999996
Average % pixels with error< 4 :  55.26519999999999
Average runtime (seconds) :  389.4
